In [1]:
import argparse
import configparser
import errno
import json
import os
import pprint
import time
import typing
from datetime import datetime, timedelta, timezone

import requests

In [2]:
def get_conversations_history(
        token: str,
        channel: str,
        latest: float = 0,
        limit: int = 100):

    Conversations_History_API_URL = "https://slack.com/api/conversations.history"

    params = {
        'token': token,
        'channel': channel,
        "limit": limit,
        "latest": latest}
    r = requests.get(Conversations_History_API_URL, params=params)
    json_data = r.json()
    if json_data.get('ok', False) is False:
        print(json_data)
        raise Exception('FAILED TO GET CONVERSATIONS HISTORY!')
    else:
        return json_data


def return_timestamp_movedbyday(diff: int = -1):
    JST = timezone(timedelta(hours=+9), 'JST')
    td_two_days_before = timedelta(days=diff)
    today_at_midnight = datetime.now(JST).replace(
        hour=0, minute=0, second=0, microsecond=0)
    two_days_before = today_at_midnight + td_two_days_before
    return two_days_before.timestamp()


def delete_message(token: str, channel: str, ts: float):
    Chat_Delete_API_URL = "https://slack.com/api/chat.delete"

    params = {'token': token, 'channel': channel, "ts": ts}
    r = requests.get(Chat_Delete_API_URL, params=params)
    json_data = r.json()

    if json_data.get('ok', False) is False:
        print('message の削除に失敗')
        print(json_data)

In [3]:
config_ini = configparser.ConfigParser()
config_ini_path = 'config.ini'

if not os.path.exists(config_ini_path):
    raise FileNotFoundError(
        errno.ENOENT, os.strerror(
            errno.ENOENT), config_ini_path)

config_ini.read(config_ini_path, encoding='utf-8')

read_default = config_ini['DEFAULT']
Token = read_default.get('OAuth_Token')
CHANNEL_ID = read_default.get('CHANNEL_ID')
BOT_ID = read_default.get('BOT_ID')


In [11]:
json_data = get_conversations_history(
    Token, CHANNEL_ID, return_timestamp_movedbyday(-10))

In [12]:
cnt = 0

for i in json_data["messages"]:
    if 'bot_id' in i:
        if i["bot_id"] == BOT_ID:
            # delete_message(Token, CHANNEL_ID, i["ts"])
            cnt += 1
            pprint.pprint(i)
print(f"{cnt} msg del done")

0 msg del done


In [8]:
json_data

{'ok': True,
 'latest': '1593097200.000000',
 'messages': [{'type': 'message',
   'subtype': 'bot_message',
   'text': ':wave: *Welcome to the Google Calendar app for Slack!*',
   'ts': '1592474979.000200',
   'username': 'Google Calendar',
   'bot_id': 'BJCGHAALB',
   'attachments': [{'fallback': '268x149px image',
     'image_url': 'https://a.slack-edge.com/interop/prod/img/calapp/img-status-gcal.png',
     'image_width': 268,
     'image_height': 149,
     'image_bytes': 18548,
     'text': '• Automatically update your Slack status based on your calendar.\n• Join Google Hangouts, Zoom or Webex video call right from the event reminder.\n• Respond to event invites &amp; get notified of time or location updates, right in Slack.\n',
     'title': 'With this app, you can...\n',
     'id': 1,
     'mrkdwn_in': ['text', 'pretext']},
    {'text': '*Connect your account to get started.*',
     'id': 2,
     'color': 'E8912D',
     'actions': [{'id': '1',
       'text': 'Connect Account',
   